<a href="https://colab.research.google.com/github/onghuiyi95/tdx2pine/blob/main/klse%20bbd%20screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[link text](https://)A demo Colab Notebook for my article: 
https://levelup.gitconnected.com/automate-your-stock-screening-using-python-9107dda724c3

In [1]:
# install required libraries (on colab)
!pip install yfinance
!pip install bs4
!pip install requests
# import required libraries 
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import email
import pandas as pd

     |████████████████████████████████| 6.3 MB 8.2 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
def get_stock_list():
  # this is the website we're going to scrape from
  url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
  response = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find(id = "MainContent2_tbAllStock")
  # return the result (only ticker code) in a list
  stock_list = table.find_all('a')
  return [stock_code.get('href')[-4:] for stock_code in stock_list]

def get_stock_price(code):
  # you can change the start date
  data = yf.download(code, start="2021-01-01", threads= False)
  return data

def add_EMA(price, day):
  return price.ewm(span=day).mean()

import numpy as np; import pandas as pd

def SMA(S, N, M=1):       #中国式的SMA,至少需要120周期才精确 (雪球180周期)    alpha=1/(1+com)    
    return pd.Series(S).ewm(alpha=M/N,adjust=True).mean().values           #com=N-M/M

def REF(S, N=1):          #对序列整体下移动N,返回序列(shift后会产生NAN)    
    return pd.Series(S).shift(N).values  

def ABS(S):      return np.abs(S)            #返回N的绝对值

def MAX(S1,S2):  return np.maximum(S1,S2)    #序列max

def EMA(S,N):             #指数移动平均,为了精度 S>4*N  EMA至少需要120周期     alpha=2/(span+1)    
    return pd.Series(S).ewm(span=N, adjust=False).mean().values    

def LLV(S,N):             # LLV(C, 5)  # 最近5天收盘最低价     
    return pd.Series(S).rolling(N).min().values    

def IF(S_BOOL,S_TRUE,S_FALSE):   #序列布尔判断 return=S_TRUE if S_BOOL==True  else  S_FALSE
    return np.where(S_BOOL, S_TRUE, S_FALSE)

def add_BBD(stock_code):
  AL=(close+low+high)/3
  AO=SMA(AL,5,1)-SMA(AL,21,1)
  BBD=(AO-SMA(AO, 3,1))*100
  
  return BBD


def add_BC(stock_code):
  AL=(close+low+high)/3
  N=30
  M=12
  VAR1=REF((low+open+close+high)/4,1) 
  VAR2=SMA(ABS(low-VAR1),13,1)/SMA(MAX(low-VAR1,0),10,1)
  VAR3=EMA(VAR2,10)
  VAR4=LLV(low,33)
  VAR5=EMA(IF(low<=VAR4,VAR3,0),3)
  LC=REF(close,1)
  RSI1=SMA(MAX(close-LC,0),12,1)/SMA(ABS(close-LC),12,1)*100
  BCa=IF(VAR5>REF(VAR5,1),VAR5,0)  ##and RSI<30,VAR5,0)
  BC=IF(RSI1<30,BCa,0)
   
  return BC

def add_STOCH(close, low, high, period, k, d=0): 
    STOCH_K = ((close - low.rolling(window=period).min()) / (high.rolling(window=period).max() - low.rolling(window=period).min())) * 100
    STOCH_K = STOCH_K.rolling(window=k).mean()
    if d == 0:
      return STOCH_K
    else:
      STOCH_D = STOCH_K.rolling(window=d).mean()
      return STOCH_D

def check_bounce_EMA(stock):
  candle1 = stock.iloc[-1]   #选取今天资料
  candle2 = stock.iloc[-2]   #选取昨天资料
  cond1 = candle1['BC'] > 0
  cond2 = candle1['BBD'] > 0 and candle2['BBD'] < 0
  cond3 = candle2['Low'] < candle2['EMA50'] and \
          candle2['Close'] > candle2['EMA50'] and \
          candle1['Low'] > candle1 ['EMA50']
  return cond1 #and cond2 # and cond3

# a list to store the screened results
screened_list = [] 
# get the full stock list
stock_list = get_stock_list()

for stock_code in stock_list:

  print(stock_code) # remove this if you dont want the ticker to be printed
  try: 
    # Step 1: get stock price for each stock
    stock = get_stock_price(stock_code + ".KL") # edit/remove ".KL" for other exchange market
    ##price_chart_df.to_csv(stock_code+".csv") 
    #stock=pd.read_csv(stock_code+".csv")
    # Step 2: add technical indicators (in this case EMA)
    close = stock['Close']
    low = stock['Low']
    open = stock['Open']
    high = stock['High']
    stock['BBD'] = add_BBD(stock_code)
    stock['BC'] = add_BC(stock_code)
    stock['EMA50'] = add_EMA(close,50)
    stock['EMA100'] = add_EMA(close,100)
    stock['STOCH_%K(5,3,3)'] = add_STOCH(close, low, high, 5, 3)
    stock['STOCH_%D(5,3,3)'] = add_STOCH(close, low, high, 5, 3, 3)

    # if all 3 conditions are met, add stock into screened list
    if check_bounce_EMA(stock):
      screened_list.append(stock_code)
      print(screened_list)
  except Exception as e:
    print(e)

print(screened_list)

## this cell may take a few minutes to finish

0012
[*********************100%***********************]  1 of 1 completed
7054
[*********************100%***********************]  1 of 1 completed
5238
[*********************100%***********************]  1 of 1 completed
00EA


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 00EA.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
7086
[*********************100%***********************]  1 of 1 completed
7131
[*********************100%***********************]  1 of 1 completed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide



7120


...

single positional indexer is out-of-bounds
38EA
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 38EA.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
3047
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3047.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
3048
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3048.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
5299
[*********************100%***********************]  1 of 1 completed
0238
[*********************100%***********************]  1 of 1 completed
3049


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3049.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
0239
[*********************100%***********************]  1 of 1 completed
3050
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3050.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
3051


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3051.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
['0122', '5099', '2682', '0048', '7214', '0068', '5258', '6998', '5069', '5100', '7209', '7277', '5908', '7198', '5228', '7094', '9318', '5592', '0078', '1147', '0082', '7676', '5182', '5095', '5160', '5614', '9393', '4723', '0111', '0170', '3565', '8303', '7164', '7130', '5172', '3174', '7006', '9628', '5143', '1198', '4936', '1171', '0167', '4502', '1694', '0155', '7219', '8141', '6548', '3913', '0026', '1724', '4634', '0007', '7544', '9113', '4286', '5517', '6084', '7186', '4405', '2054', '0084', '6378', '7374', '7285', '7173', '0140', '7240', '5016', '0165', '5048', '6742', '2283', '5252', '5256', '5264', '0193', '5288', '0203', '0227', '0229']


In [ ]:
def get_stock_list():
  # this is the website we're going to scrape from
  url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
  response = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find(id = "MainContent2_tbAllStock")
  # return the result (only ticker code) in a list
  stock_list = table.find_all('a')
  return [stock_code.get('href')[-4:] for stock_code in stock_list]

def get_stock_price(code):
  # you can change the start date
  data = yf.download(code, start="2021-01-01", threads= False)
  return data

def add_EMA(price, day):
  return price.ewm(span=day).mean()

import numpy as np; import pandas as pd

def SMA(S, N, M=1):       #中国式的SMA,至少需要120周期才精确 (雪球180周期)    alpha=1/(1+com)    
    return pd.Series(S).ewm(alpha=M/N,adjust=True).mean().values           #com=N-M/M

def REF(S, N=1):          #对序列整体下移动N,返回序列(shift后会产生NAN)    
    return pd.Series(S).shift(N).values  

def ABS(S):      return np.abs(S)            #返回N的绝对值

def MAX(S1,S2):  return np.maximum(S1,S2)    #序列max

def EMA(S,N):             #指数移动平均,为了精度 S>4*N  EMA至少需要120周期     alpha=2/(span+1)    
    return pd.Series(S).ewm(span=N, adjust=False).mean().values    

def LLV(S,N):             # LLV(C, 5)  # 最近5天收盘最低价     
    return pd.Series(S).rolling(N).min().values    

def IF(S_BOOL,S_TRUE,S_FALSE):   #序列布尔判断 return=S_TRUE if S_BOOL==True  else  S_FALSE
    return np.where(S_BOOL, S_TRUE, S_FALSE)

def add_BBD(stock_code):
  AL=(close+low+high)/3
  AO=SMA(AL,5,1)-SMA(AL,21,1)
  BBD=(AO-SMA(AO, 3,1))*100
  
  return BBD


def add_BC(stock_code):
  AL=(close+low+high)/3
  N=30
  M=12
  VAR1=REF((low+open+close+high)/4,1) 
  VAR2=SMA(ABS(low-VAR1),13,1)/SMA(MAX(low-VAR1,0),10,1)
  VAR3=EMA(VAR2,10)
  VAR4=LLV(low,33)
  VAR5=EMA(IF(low<=VAR4,VAR3,0),3)
  LC=REF(close,1)
  RSI1=SMA(MAX(close-LC,0),12,1)/SMA(ABS(close-LC),12,1)*100
  BCa=IF(VAR5>REF(VAR5,1),VAR5,0)  ##and RSI<30,VAR5,0)
  BC=IF(RSI1<30,BCa,0)
   
  return BC

def add_STOCH(close, low, high, period, k, d=0): 
    STOCH_K = ((close - low.rolling(window=period).min()) / (high.rolling(window=period).max() - low.rolling(window=period).min())) * 100
    STOCH_K = STOCH_K.rolling(window=k).mean()
    if d == 0:
      return STOCH_K
    else:
      STOCH_D = STOCH_K.rolling(window=d).mean()
      return STOCH_D

def check_bounce_EMA(stock):
  candle1 = stock.iloc[-1]   #选取今天资料
  candle2 = stock.iloc[-2]   #选取昨天资料
  cond1 = candle1['BC'] > 0
  cond2 = candle1['BBD'] > 0 and candle2['BBD'] < 0
  cond3 = candle2['Low'] < candle2['EMA50'] and \
          candle2['Close'] > candle2['EMA50'] and \
          candle1['Low'] > candle1 ['EMA50']
  return cond1 #and cond2 # and cond3

# a list to store the screened results
screened_list = [] 
# get the full stock list
stock_list = get_stock_list()

for stock_code in stock_list:

  print(stock_code) # remove this if you dont want the ticker to be printed
  try: 
    # Step 1: get stock price for each stock
    stock = get_stock_price(stock_code + ".KL") # edit/remove ".KL" for other exchange market
    ##price_chart_df.to_csv(stock_code+".csv") 
    #stock=pd.read_csv(stock_code+".csv")
    # Step 2: add technical indicators (in this case EMA)
    close = stock['Close']
    low = stock['Low']
    open = stock['Open']
    high = stock['High']
    stock['BBD'] = add_BBD(stock_code)
    stock['BC'] = add_BC(stock_code)
    stock['EMA50'] = add_EMA(close,50)
    stock['EMA100'] = add_EMA(close,100)
    stock['STOCH_%K(5,3,3)'] = add_STOCH(close, low, high, 5, 3)
    stock['STOCH_%D(5,3,3)'] = add_STOCH(close, low, high, 5, 3, 3)

    # if all 3 conditions are met, add stock into screened list
    if check_bounce_EMA(stock):
      screened_list.append(stock_code)
      print(screened_list)
  except Exception as e:
    print(e)

print(screened_list)

## this cell may take a few minutes to finish

0012
[*********************100%***********************]  1 of 1 completed
7054
[*********************100%***********************]  1 of 1 completed
5238
[*********************100%***********************]  1 of 1 completed
00EA


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 00EA.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
7086
[*********************100%***********************]  1 of 1 completed
7131
[*********************100%***********************]  1 of 1 completed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide



7120


...

single positional indexer is out-of-bounds
38EA
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 38EA.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
3047
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3047.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
3048
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3048.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
5299
[*********************100%***********************]  1 of 1 completed
0238
[*********************100%***********************]  1 of 1 completed
3049


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3049.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
0239
[*********************100%***********************]  1 of 1 completed
3050
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3050.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
3051


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3051.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds
['0122', '5099', '2682', '0048', '7214', '0068', '5258', '6998', '5069', '5100', '7209', '7277', '5908', '7198', '5228', '7094', '9318', '5592', '0078', '1147', '0082', '7676', '5182', '5095', '5160', '5614', '9393', '4723', '0111', '0170', '3565', '8303', '7164', '7130', '5172', '3174', '7006', '9628', '5143', '1198', '4936', '1171', '0167', '4502', '1694', '0155', '7219', '8141', '6548', '3913', '0026', '1724', '4634', '0007', '7544', '9113', '4286', '5517', '6084', '7186', '4405', '2054', '0084', '6378', '7374', '7285', '7173', '0140', '7240', '5016', '0165', '5048', '6742', '2283', '5252', '5256', '5264', '0193', '5288', '0203', '0227', '0229']


In [5]:
print(screened_list)

['0122', '5099', '2682', '0048', '7214', '0068', '5258', '6998', '5069', '5100', '7209', '7277', '5908', '7198', '5228', '7094', '9318', '5592', '0078', '1147', '0082', '7676', '5182', '5095', '5160', '5614', '9393', '4723', '0111', '0170', '3565', '8303', '7164', '7130', '5172', '3174', '7006', '9628', '5143', '1198', '4936', '1171', '0167', '4502', '1694', '0155', '7219', '8141', '6548', '3913', '0026', '1724', '4634', '0007', '7544', '9113', '4286', '5517', '6084', '7186', '4405', '2054', '0084', '6378', '7374', '7285', '7173', '0140', '7240', '5016', '0165', '5048', '6742', '2283', '5252', '5256', '5264', '0193', '5288', '0203', '0227', '0229']
